In [84]:
import pandas as pd
import numpy as np
import plotly.express as px
import requests
import dash
from dash import dcc, html, Input, Output, State
import dash_bootstrap_components as dbc
from datetime import datetime

In [85]:
api_key = 'f19b4887-c766-4889-b1b5-913d7ec9b234'
url_assets = "https://api.coincap.io/v2/assets"

headers = {
    "Authorization": f"Bearer {api_key}"
}

In [86]:
def get_assets():
    response = requests.get(url_assets, headers=headers)
    data = response.json()
    assets = data['data']
    return assets

In [87]:
assets = get_assets()

In [88]:
def get_asset_history(asset_id, start_date, end_date):
    url_history = f"https://api.coincap.io/v2/assets/{asset_id}/history?interval=d1&start={start_date}&end={end_date}"
    response = requests.get(url_history, headers=headers)
    data = response.json()
    history = data['data']
    df = pd.DataFrame(history)
    df['date'] = pd.to_datetime(df['time'], unit='ms')
    df.drop(columns=['time'], inplace=True)
    return df

In [89]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = dbc.Container(
    [
        dbc.Row(
            [
                dbc.Col(
                    dbc.Card(
                        dbc.CardBody(
                            [
                                html.Label('Select an asset'),
                                dcc.Dropdown(
                                    id='asset-dropdown',
                                    options=[{'label': asset['id'], 'value': asset['id']} for asset in assets],
                                    value=assets[0]['id'] if assets else None,
                                    className="mb-2 w-100"
                                ),
                                
                                html.Br(),
                                
                                html.Label('Date From '),
                                dcc.Input(
                                    id='date-from',
                                    type='text',
                                    value='05.05.2022',
                                    style={
                                        "border": "1px solid #cccccc",
                                        "border-radius": "0.25rem",
                                    },
                                    className="form-control mb-2 w-100"
                                ),
                                
                                
                                
                                html.Label('Date to '),
                                dcc.Input(
                                    id='date-to',
                                    type='text',
                                    value='05.05.2023',
                                    style={
                                        "border": "1px solid #cccccc",
                                        "border-radius": "0.25rem"
                                    },
                                    className="form-control mb-2 w-100"
                                ),

                                html.Br(),
                                
                                dbc.Button(
                                    'Show',
                                    id='submit-button',
                                    n_clicks=0,
                                    className="btn btn-primary mb-2 w-100"
                                )
                            ]
                        ),
                        className="mb-2 w-100",
                        style={
                                "border": "0px",
                                "box-shadow": "0 0px 8px rgba(0, 0, 0, 0.1)",
                                "border-radius": "15px",
                            }
                    ),
                    width=3,
                ),
                
                dbc.Col(
                    dbc.Card(
                        dbc.CardBody(
                            [
                                dcc.Graph(
                                    id='price-graph',
                                    config={'displayModeBar': False}
                                )
                            ]
                        ),
                        className="mb-2 w-100",
                        style={
                                "border": "0px",
                                "box-shadow": "0 0px 8px rgba(0, 0, 0, 0.1)",
                                "border-radius": "15px",
                            }
                    ),
                    width=9
                )
            ],
            className="mb-2",
            align='center',
            style={'min-height': '100vh'}
        )
    ],
    fluid=True,
    style={
        'background-color': '#f3f3f3',
        'min-height': '80vh',
    }
)

@app.callback(
    Output('price-graph', 'figure'),
    Input('submit-button', 'n_clicks'),
    State('asset-dropdown', 'value'),
    State('date-from', 'value'),
    State('date-to', 'value')
)
def update_graph(n_clicks, selected_asset, date_from_str, date_to_str):
    date_from = datetime.strptime(date_from_str, '%d.%m.%Y').strftime('%Y-%m-%d')
    date_to = datetime.strptime(date_to_str, '%d.%m.%Y').strftime('%Y-%m-%d')
    
    start_date = int(pd.Timestamp(date_from).timestamp() * 1000)
    end_date = int(pd.Timestamp(date_to).timestamp() * 1000)
    df = get_asset_history(selected_asset, start_date, end_date)
    df.priceUsd = pd.to_numeric(df.priceUsd, errors='coerce')
    fig = px.bar(
        df,
        x='date',
        y='priceUsd',
        labels={'priceUsd': 'Price', 'date': 'Date'},
        hover_name='date',
        title='Price and date ratio',
        color_discrete_sequence=['#0090ff'],
    )
    fig.update_layout(
        plot_bgcolor='white',
    )
    return fig

if __name__ == '__main__':
    app.run_server(debug=False)